In [11]:
import os
import pickle
import numpy as np
from scipy.io import loadmat

import scfw.cov_estimation as ce
from scfw.frank_wolfe import frank_wolfe
from scfw.scopt import scopt
from scfw.prox_grad import prox_grad

In [12]:
data_folder = '../data/'
problem_name = 'cov_estimation'
results = {problem_name: {}}
results_folder = './results/'
results_file = os.path.join(results_folder, 'cov_estimation', problem_name + '.pckl')
if os.path.isfile(results_file):
    with open(results_file, "rb") as f:
        try:
            results = pickle.load(f)
        except Exception:
            pass 

In [13]:
def sample_l1_point(r, ndim):
    vec = np.random.rand(ndim,)
    vec /= np.linalg.norm(vec, ord=1)
    return vec

In [14]:
# load data
data = loadmat(os.path.join(data_folder, problem_name + '.mat'))
    
A = data['S']
p = A.shape[0]
r = 1300
X_0 = r * np.eye(p) / p
#X_0 = np.diag(sample_l1_point(r, p))

FileNotFoundError: [Errno 2] No such file or directory: '../data/cov_estimation.mat'

## Parameters initialization

In [5]:
#first set of parameters
Mf = 2
nu = 3
#running parameters

terminate_tol = 1e-15

FW_params={
    'iter_FW': 30000,
    'line_search_tol': 1e-5,
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 100,
    'Lest': 'estimate', #estimate L
    'use_two_phase': True,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 100,
    #Conjugate Gradient Parameters
    'conj_grad_tol': 1e-2,
    'conj_grad_iter': 100,
}

## Functions

In [6]:
func_x = lambda X: ce.cov_val(A, X)
func_beta = lambda X, S, beta, extra_param, extra_param_s: ce.cov_val(A, (1 - beta) * X + beta * S)
grad_x = lambda X, inv_X: ce.cov_grad(A, X, inv_X)
grad_beta = lambda X, S, beta, inv_X, inv_S: ce.cov_grad(A, (1 - beta) * X + beta * S)
hess_mult_x = lambda S, inv_X: ce.cov_hess_mult(S, inv_X)
hess_mult_vec_x = lambda S, inv_X: ce.cov_hess_mult_vec(S, inv_X)
#extra_func = lambda X: np.linalg.eigvalsh(X)
extra_func = lambda X: X
linear_oracle = lambda grad: ce.linear_oracle(grad, r)
prox_func = lambda X, L: ce.projection(X, r) #used for SCOPT

## Run FW

In [11]:
#run_alpha_policies = ['standard', 'line_search', 'backtracking', 'sc', 'sc_backtracking', 'sc_hybrid']
run_alpha_policies = ['line_search', 'backtracking', 'sc']

for policy in run_alpha_policies:
    x, alpha_hist, Gap_hist, Q_hist, time_hist  = frank_wolfe(func_x,
                       func_beta,                                       
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       linear_oracle,                                                    
                       X_0,
                       FW_params,
                       #hess=hess_x, 
                       #lloo_oracle=llo_oracle,                                                 
                       alpha_policy=policy,                                                    
                       eps=terminate_tol, 
                       print_every=1000, 
                       debug_info=False)
    
    results[problem_name + '.mat'][policy] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Gap_hist': Gap_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    }
#    with open('./results/cov_estimation/lymph.pckl', 'wb') as f:
#        pickle.dump(results, f)

********* Algorithm starts *********
iter=1, stepsize=2.93e-03, criterion=1.00e-05, upper_bound=833.279397234206, lower_bound=-1177.7958435120934, real_Gap=2.01e+03, f_val=833.279397234206
iter=1000, stepsize=3.40e-04, criterion=1.00e-05, upper_bound=365.6237479710911, lower_bound=-92.77553456170091, real_Gap=4.58e+02, f_val=365.6237479710911
iter=2000, stepsize=1.74e-04, criterion=1.00e-05, upper_bound=330.51029930742243, lower_bound=75.91857969459488, real_Gap=2.55e+02, f_val=330.51029930742243
iter=3000, stepsize=6.87e-05, criterion=1.00e-05, upper_bound=318.120683268992, lower_bound=134.77031530118194, real_Gap=1.83e+02, f_val=318.120683268992
iter=4000, stepsize=5.34e-05, criterion=1.00e-05, upper_bound=310.7950618640304, lower_bound=164.86797960326018, real_Gap=1.46e+02, f_val=310.7950618640304
iter=5000, stepsize=5.34e-05, criterion=1.00e-05, upper_bound=305.91400828774954, lower_bound=182.46013431462598, real_Gap=1.23e+02, f_val=305.91400828774954
iter=6000, stepsize=7.47e-05, 

In [10]:
x, alpha_hist, Q_hist, time_hist = scopt(func_x,
          grad_x,
          hess_mult_x,
          hess_mult_vec_x,
          Mf,
          nu,
          prox_func,
          X_0,  
          sc_params,                                              
          eps=terminate_tol,                                              
          print_every=1)
    
results[problem_name + '.mat']['scopt'] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    }

Fista err = 1.924e-13; Subiter =   2; subproblem converged!
iter =    1, stepsize = 3.329e-02, rdiff = 1.198e+00 , f = 833.279
iter =    2, stepsize = 3.439e-02, rdiff = 1.065e+00 , f = 775.809
iter =    3, stepsize = 3.606e-02, rdiff = 1.038e+00 , f = 724.828
iter =    4, stepsize = 3.607e-02, rdiff = 1.037e+00 , f = 677.574
iter =    5, stepsize = 3.583e-02, rdiff = 1.035e+00 , f = 633.844
iter =    6, stepsize = 3.561e-02, rdiff = 1.031e+00 , f = 593.821
iter =    7, stepsize = 3.588e-02, rdiff = 1.012e+00 , f = 557.127
iter =    8, stepsize = 3.678e-02, rdiff = 9.736e-01 , f = 523.302
iter =    9, stepsize = 3.735e-02, rdiff = 9.506e-01 , f = 491.984
iter =   10, stepsize = 3.789e-02, rdiff = 9.310e-01 , f = 463.356
iter =   11, stepsize = 3.955e-02, rdiff = 8.829e-01 , f = 437.344
iter =   12, stepsize = 4.292e-02, rdiff = 8.088e-01 , f = 413.752
iter =   13, stepsize = 4.771e-02, rdiff = 7.144e-01 , f = 392.557
iter =   14, stepsize = 5.302e-02, rdiff = 6.301e-01 , f = 373.717
it

KeyError: 'Lymph'

In [7]:
prox_params={
        #parameters for SCOPT
        'iter_prox': 5000,
        'Lest': 'estimate',#,'estimate', #estimate L
        'bb_type': 2,
        #FISTA parameters
        'fista_type': 'fista',
        'fista_tol': 1e-5,
        'fista_iter': 1000,
        'btk_iters': 100,
        'backtracking': True
    }

In [8]:
x, alpha_hist, Q_hist, time_hist = prox_grad(func_x,
            grad_x,
            prox_func,
            Mf,
            X_0,
            prox_params,
            eps=terminate_tol,
            print_every=10)

 , f = 289.116
iter = 2030, stepsize = 1.000e+00, rdiff = 1.489e-10 , f = 289.116
iter = 2040, stepsize = 1.000e+00, rdiff = 1.355e-09 , f = 289.116
iter = 2050, stepsize = 1.000e+00, rdiff = 9.821e-11 , f = 289.116
iter = 2060, stepsize = 1.000e+00, rdiff = 2.645e-11 , f = 289.116
iter = 2070, stepsize = 1.000e+00, rdiff = 2.644e-10 , f = 289.116
iter = 2080, stepsize = 1.000e+00, rdiff = 6.251e-10 , f = 289.116
iter = 2090, stepsize = 1.000e+00, rdiff = 2.069e-10 , f = 289.116
iter = 2100, stepsize = 1.000e+00, rdiff = 5.982e-10 , f = 289.116
iter = 2110, stepsize = 1.000e+00, rdiff = 4.139e-10 , f = 289.116
iter = 2120, stepsize = 1.000e+00, rdiff = 1.662e-10 , f = 289.116
iter = 2130, stepsize = 1.000e+00, rdiff = 2.633e-11 , f = 289.116
iter = 2140, stepsize = 1.000e+00, rdiff = 1.147e-11 , f = 289.116
iter = 2150, stepsize = 1.000e+00, rdiff = 4.663e-10 , f = 289.116
iter = 2160, stepsize = 1.000e+00, rdiff = 4.998e-12 , f = 289.116
iter = 2170, stepsize = 1.000e+00, rdiff = 1.17

In [9]:
results[problem_name + '.mat']['prox_grad'] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    }

In [10]:
with open('./results/cov_estimation/lymph.pckl', 'wb') as f:
    pickle.dump(results, f)

In [7]:
x, alpha_hist, Gap_hist, Q_hist, time_hist  = frank_wolfe(func_x,
                       func_beta,                                       
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       linear_oracle,                                                    
                       X_0,
                       FW_params,
                       #hess=hess_x, 
                       #lloo_oracle=llo_oracle,                                                 
                       alpha_policy='line_search',                                                    
                       eps=terminate_tol, 
                       print_every=1, 
                       debug_info=False)
                       

********* Algorithm starts *********
min_eig: -293.500000
beta_max: 0.003396
0.0033889536353396274
-652.2146507666092
iter=1, stepsize=3.39e-03, criterion=5.82e-02, upper_bound=833.279397234206, lower_bound=-1177.7958435120934, real_Gap=2.01e+03, f_val=833.279397234206
min_eig: 0.000000
beta_max: 1.000000
0.9999923705054695
-2.2108780343174272
iter=2, stepsize=1.00e+00, criterion=5.82e-02, upper_bound=832.010083035731, lower_bound=-1177.7958435120934, real_Gap=2.01e+03, f_val=832.010083035731
min_eig: 0.000000
beta_max: 1.000000
0.9999923705054695
-1299.9900984965147
iter=3, stepsize=1.00e+00, criterion=5.82e-02, upper_bound=832.010083035731, lower_bound=-1177.7958435120934, real_Gap=2.01e+03, f_val=7734.018144452073
min_eig: 0.000000
beta_max: 1.000000
0.9999923705054695
-1299.9900816572379
iter=4, stepsize=1.00e+00, criterion=5.82e-02, upper_bound=832.010083035731, lower_bound=-1177.7958435120934, real_Gap=2.01e+03, f_val=14620.980842775627
min_eig: 0.000000
beta_max: 1.000000
0.9999

KeyboardInterrupt: 

In [12]:
min_eig, _ = scipy.linalg.eigh(X_0)
print(min(min_eig))

2.2146507666098807


In [11]:
import scipy

In [23]:
a = np.array([[0, -10, 0], [-10, 0, 0], [0, 0, 0]])

In [24]:
min_eig, _ = scipy.linalg.eigh(a)
print(min(min_eig))

-9.99999999999999
